<a href="https://colab.research.google.com/github/sandvenu/GENAI-/blob/main/MongoDB_and_PineconeRealtime_AI_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Soource the required librares first

In [ ]:
!pip install pinecone-client  pymongo  transformers

In [27]:
! pip install "pymongo[srv]"==3.11

Establish the Mongo DB connection

In [28]:

from pymongo.mongo_client import MongoClient

uri = "mongodb+srv://##########@cluster0.itasbqv.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


Establsih the Pine cone connection for Vector DB

In [29]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="39f2e44a-de8c-4b87-b631-b50cfca30590")

In [30]:
pc.create_index(
    name="quickstart4",
    dimension=384, # Replace with your model dimensions
    metric="euclidean", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
!pip install sentence-transformers

In [32]:
from sentence_transformers import SentenceTransformer, util

In [33]:
# transformer stuff
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [34]:
db = client['mydb']
collection = db['mycollection']

Insert a record in Mongo DB

Based on create/Update/Delete records in MOngo DB update the embeddings in Pine cone DB

In [35]:
index = pc.Index("quickstart4")

In [ ]:
import threading
import time
from pymongo import MongoClient

def listen_to_changes(collection):

    try:
        print("Starting to watch the collection for changes...")
        with collection.watch(full_document='updateLookup') as stream:
            print("Change stream is now open.")
            for change in stream:
                print("Change detected:", change)
                if change['operationType'] == 'insert':
                   document = change['fullDocument']
                   # convert the document's name into an embedding
                   vector = model.encode(document['name'])
                   # Ensure the vector is a flat list of floats (and possibly convert to float64)
                   vector = vector.tolist()  # Convert from numpy array to list
                   vector = [float(x) for x in vector]  # Convert elements to float (usually float64)
                   # Prepare the data for Pinecone upsert, which requires a tuple of (id, vector)
                   # Assuming 'document['_id']' is the unique ID for the upsert operation
                   upsert_data = (str(document['_id']), vector)
                  # Insert into Pinecone
                   index.upsert([upsert_data])  # Note that upsert_data is enclosed in a list
                   # insert into pinecone
                   #index.upsert(vector)
                   #pc.upsert(index_name="quickstart3", data=vector, ids=[str(document['_id'])])
            print("Exited the change stream loop.")
        print("Change stream closed.")
    except Exception as e:
        print(f"An error occurred in the change stream: {e}")
client = MongoClient(uri)
db = client['mydb']
collection = db['mycollection']

# Start listening to change stream in a separate thread
listener_thread = threading.Thread(target=listen_to_changes, args=(collection,))
listener_thread.start()

# Sleep for a short period to ensure the listener thread is ready
time.sleep(2)

# Now the main thread is free to perform other operations on the collection
collection.insert_one({'name': 'Xouglas', 'age': 36, 'city': 'New York'})

# Wait for the listener thread to finish (it won't in this example, due to the infinite loop)
listener_thread.join()

Starting to watch the collection for changes...
Change stream is now open.
{'_id': {'_data': '82663248DB000000092B042C0100296E5A10040F523822A86D4249A07E10289236A57E463C6F7065726174696F6E54797065003C696E736572740046646F63756D656E744B65790046645F69640064663248DA2800D792E906C958000004'}, 'operationType': 'insert', 'clusterTime': Timestamp(1714571483, 9), 'wallTime': datetime.datetime(2024, 5, 1, 13, 51, 23, 630000), 'fullDocument': {'_id': ObjectId('663248da2800d792e906c958'), 'name': 'Xouglas', 'age': 36, 'city': 'New York'}, 'ns': {'db': 'mydb', 'coll': 'mycollection'}, 'documentKey': {'_id': ObjectId('663248da2800d792e906c958')}}{'_id': {'_data': '82663248DB000000092B042C0100296E5A10040F523822A86D4249A07E10289236A57E463C6F7065726174696F6E54797065003C696E736572740046646F63756D656E744B65790046645F69640064663248DA2800D792E906C958000004'}, 'operationType': 'insert', 'clusterTime': Timestamp(1714571483, 9), 'wallTime': datetime.datetime(2024, 5, 1, 13, 51, 23, 630000), 'fullDocument': {'_id